# Press the Double Arrow above to Begin, then click red pop-up to confirm

In [ ]:
display("Loading Allen Transcriptomics Single-Cell RNAseq data, retreived May 2020. Be patient, it's loading!")

In [1]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:1000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [4]:
import pandas as pd
import plotly.express as px
import ipywidgets as widgets

In [5]:

mouse_means0 = pd.read_csv('https://idk-etl-prod-download-bucket.s3.amazonaws.com/aibs_mouse_ctx-hip_smart-seq/trimmed_means.csv')
human_means0 = pd.read_csv('https://idk-etl-prod-download-bucket.s3.amazonaws.com/aibs_human_ctx_smart-seq/trimmed_means.csv')

# Get all gene and cell type names: HUMAN
human_idx = list(human_means0.columns.values)
human_listOfColumnNames = list(human_means0['cluster_label'])
# Get all gene and cell type: MOUSE
mouse_idx = list(mouse_means0.columns.values)
mouse_listOfColumnNames = list(mouse_means0['cluster_label'])


In [6]:

#combine clusters into higher_clusters
newlist =[]
for i in mouse_listOfColumnNames:
    i = str(i)
    head, sep, tail = i.partition('_') 
    newlist.append(tail) 
mouse_means0['higher_cluster'] = newlist



#list what cell clusters count as which cell types
excite_list = ['CA1','CA1-ProS','CA2-IG-FC','CA3','CT SUB','DG', 'L2 IT ENTl',
 'L2 IT ENTm','L2 IT HATA','L2 IT PAR','L2 IT ProS','L2 IT RSP-ACA','L2 IT RSPv',
 'L2/3 IT APr','L2/3 IT CTX','L2/3 IT ENTl','L2/3 IT PPP','L2/3 IT TPE',
 'L3 IT ENTl','L3 IT ENTm','L3 RSP-ACA','L4/5 IT CTX','L4/5 IT TPE-ENT',
 'L5 IT CTX','L5 IT TPE-ENT','L5 NP CTX','L5 PPP','L5 PT CTX','L5 PT RSP-ACA',
 'L5/6 IT CTX', 'L6 Car3', 'L6 CT CTX','L6 CT ENTm','L6 IT CTX','L6 IT ENTl',
 'L6 IT RHP','L6 NP CT CTX','L6b CTX','L6b RHP','L6b/CT ENT','Pax6','SUB','SUB-ProS', 'ProS','Ndnf HPF'
 'NP PPP','NP SUB']

non_neuron_list = [ 'VLMC', 'Oligo', 'Micro', 'Endo', 'Astro', 'CR', 'OPC', 'Peri',  'PVM', 'SMC', 'v3d']

inhib_list =  ['Lamp5 Lhx6','Lamp5','Meis2','Pvalb Vipr2','Pvalb','Sncg','Sst Chodl','Sst','Vip']

all_list = excite_list + non_neuron_list + inhib_list

#add cell type data to frame
mega_grps = []
for i in mouse_means0["higher_cluster"]:
    if i in excite_list :
        mega_grps.append("Excitatory")
    if i in inhib_list :
        mega_grps.append("Inhibitory")
    if i in non_neuron_list :
        mega_grps.append("Non-Neuronal")
    if i not in all_list :
        mega_grps.append("Unknown")
mouse_means0["Cell_Type"] = mega_grps


In [7]:
cells =[]
genes = []
graph_out1 = widgets.Output()  
graph_out2 = widgets.Output()  
def on_go_button_clicked(go_button):

#Mark errors
    if len(genes) < 1:
        with outerrors:
            print("Choose at least one Gene")
    if len(list(Cells_drop.value)) < 1:
        with outerrors:
            print("Choose at least one Cell Cluster")
            
## restructure data frame to include selected types only 
    if len(genes) >= 1:
         if len(list(Cells_drop.value)) >= 1:
            data_gene_cell = mouse_means0[mouse_means0["higher_cluster"].isin(list(Cells_drop.value))]
            genes.append("cluster_label")
            genes.append("higher_cluster")
            genes.append("Cell_Type")

            data_gene_cell2 = data_gene_cell[genes]
            data_gene_cell2 = data_gene_cell.reset_index()
                        #data_gene_cell2
            mouse_means = pd.melt(data_gene_cell2, id_vars =['cluster_label', 'higher_cluster','Cell_Type'],
                                          var_name ='GeneID', value_name ='Expression'
                                     )
            
            
            
   ###Graph it         
            import plotly.graph_objects as go
            fig = px.box(mouse_means, x="Cell_Type", y="Expression", color = "Cell_Type", 
                    facet_col = "GeneID",
                    hover_name = "Cell_Type",
                    hover_data = ["Cell_Type", "cluster_label"]
                    )
            #fig.update_layout(barmode='group')
            with graph_out1:
                    fig.show()

            ## print table
            fig2 = go.Figure(data=[go.Table(
            header=dict(values=list(data_gene_cell2.columns),
                    fill_color='paleturquoise',
                    align='left'),
            cells=dict(values=[data_gene_cell2.GeneID, data_gene_cell2.Cluster, data_gene_cell2.Expression,
                              data_gene_cell2.Cell_Type, data_gene_cell2.Age, data_gene_cell2.Category],
                    fill_color='lavender',
                    align='left'))
                ])
            with graph_out2:
                    fig2.show()

In [8]:
##define GUI  
def choose_gene(change):
    if dropdown.value not in genes:
        if dropdown.value != '':
            genes.append(dropdown.value)
            with outgenes:
                display(dropdown.value)
                
                
spacer = widgets.Label('')
genetitle = widgets.Label('Select Genes')

dropdown =  widgets.Combobox(
    options= mouse_idx,
    description='Gene',
    placeholder='Type here',
    layout={'width': '80%'})

   
outgenes = widgets.Output()   
outgraph = widgets.Output(layout={'border': '1px solid black'})  

        
gene_button = widgets.Button(description = 'Enter Gene', 
                button_style = '', 
                display='flex',
                flex_flow='column',
                align_items='stretch')

gene_button.on_click(choose_gene)

celllabel = widgets.Label('Cell Clusters:')
Cells_drop =widgets.SelectMultiple(
    options=all_list,
    rows=8,
    layout={'width': 'max-content'}) 


go_button = widgets.Button(description = 'Produce Output', 
                           button_style = '', 
                            display='flex',
                            flex_flow='column',
                            align_items='stretch')
go_button.on_click(on_go_button_clicked)

gene_button.observe(dropdown.value)
go_button.observe(dropdown.value)
go_button.observe(Cells_drop.value)


graph_out1 = widgets.Output()  
graph_out2 = widgets.Output()  

In [9]:
genesbox = widgets.VBox([genetitle,dropdown, gene_button])
cellsbox = widgets.VBox([celllabel, Cells_drop])
genechoices = widgets.VBox([genesbox, outgenes,go_button])
outerrors = widgets.Output() 

graphit = widgets.VBox([graph_out1, graph_out2])
top_app = widgets.HBox([cellsbox,genechoices, outerrors])
app = widgets.VBox([top_app, graphit])


app

In [ ]:
##define GUI
table_style = {'description_width': 'initial'}
useidx = ('choose species','then genes will appear')
spacer = widgets.Label('')

from threading import Timer

from traitlets import traitlets

class LoadedButton(widgets.Button):
    """A button that can hold a value as a attribute."""

    def __init__(self, value=None, *args, **kwargs):
        super(LoadedButton, self).__init__(*args, **kwargs)
        # Create the value attribute.
        self.add_traits(value=traitlets.Any(value))
#####################################################        
genes = []
genetitle = widgets.Label('Select Genes')
dropdown =  widgets.Combobox(
    options= mouse_idx,
    description='Gene',
    placeholder='Type here',
    layout={'width': '80%'})

def delay():
    gene_button.button_style = ''
    gene_button.disabled = False
    
def choose_gene(change):
    if len(genes)<=7:
        gene_button.button_style = 'success'
        gene_button.disabled = True
        genes.append(dropdown.value)     
        d = Timer(0.5,delay)
        d.start()
    if len(genes)>7:       
        gene_button.button_style = 'warning'
        gene_button.description = "Max Genes Selected"
        gene_button.disabled = True
        
gene_button = widgets.Button(description = 'Enter Gene', button_style = '')
gene_button.on_click(choose_gene)


choicelabel = widgets.Label('Analyze species:')
msCells = widgets.Button(
    description='Mouse',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    width = '110%')

huCells = widgets.Button(
    description='Human',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    width = '110%')

#####################################################
mscelllabel = widgets.Label('Mouse Cell Clusters:')
msCells_drop =widgets.SelectMultiple(
    options=mouse_listOfColumnNames,
    disabled=True,
    rows=17,
    layout={'width': 'max-content'},
    #style=table_style
) 

hucelllabel = widgets.Label('Human Cell Clusters:')
huCells_drop =widgets.SelectMultiple(
    options=human_listOfColumnNames,
    disabled=True,
    rows=17,
    layout={'width': 'max-content'},
    #style=table_style
) 

def choose_mouse(msCells):
    huCells.disabled = True
    msCells.button_style = 'success'
    msCells_drop.disabled = False
    huCells_drop.disabled = True 
    msCells.value = "mouse"

msCells.on_click(choose_mouse)

def choose_human(huCells):
    huCells.button_style = 'success'
    msCells.disabled = True
    msCells_drop.disabled = True
    huCells_drop.disabled = False
    huCells.value = "human"
    dropdown.options = human_idx

huCells.on_click(choose_human)

msCells = LoadedButton(description="Mouse", value="not mouse")
msCells.on_click(choose_mouse)   

huCells = LoadedButton(description="Human", value="not human")
huCells.on_click(choose_human) 
dropdown.observe(choose_human, 'change')
########################################

go_button = widgets.Button(description = 'Produce Output', 
                           button_style = 'success', 
                            display='flex',
                            flex_flow='column',
                            align_items='stretch')
reset_button = widgets.Button(description = 'Reset Options',
                              button_style = 'warning',
                            display='flex',
                            flex_flow='column',
                            align_items='stretch',
                             width = 'auto')

from ipywidgets import Widget
cells =[]
def on_go_button_clicked(go_button):
    if huCells.value == 'human':
        cells.append(huCells_drop.value)
    else:
        cells.append(msCells_drop.value)
    expgraph(genes,cells)


def on_reset_button_clicked(reset_button):  
    Widget.close_all()
    
go_button.observe(huCells, 'value')   
go_button.on_click(on_go_button_clicked)
reset_button.on_click(on_reset_button_clicked)




specieschoices = widgets.VBox([choicelabel,msCells, mscelllabel, msCells_drop])
spacerhuman = widgets.VBox([spacer, huCells,hucelllabel, huCells_drop])
genesbox = widgets.VBox([genetitle,dropdown, gene_button])
buttons = widgets.HBox([go_button, spacer, reset_button])
graphchoices = widgets.VBox([spacer, spacer, genesbox,spacer, spacer, spacer, buttons])
#graphs2 = widgets.HBox([graphchoices])
####################################################
display("..............")
display("Choose your Species and Graph Type. For Expression level graphs, choose cell types to include. For Venn diagrams, all cell types will be included. Choose genes, pressing Enter after each choice. To produce graphs, press the green Produce Output button.")

In [ ]:
%matplotlib notebook

def expgraph(genes, cells): 
    if msCells.value == 'mouse':
        bigframe = mouse_means0
        cells = msCells_drop.value
    if huCells.value == 'human':
        bigframe = human_means
        cells = huCells_drop.value

    newdf = pd.DataFrame(columns = ['Cells','Genes','Expression'])
    newdf['Cells'] = cells * len(genes)
    print(newdf)

    genelist=[]
    for i in range(len(genes)):
        genea = [genes[i]]*len(cells)
        genelist.append(genea)
    flat_geneslist = [item for sublist in genelist for item in sublist]
    newdf['Genes'] = flat_geneslist

    expressions =[]
    for i in range(len(genes)*len(cells)):
        value1 = bigframe.loc[newdf['Genes'][i], newdf['Cells'][i]]
        expressions.append(value1)
    newdf['Expression'] = expressions

    if len(newdf['Expression'])>0:
        maxm = max(newdf["Expression"])
    

    #newdf.to_excel(excelname.value) 
    
    sns.set_palette("colorblind", 10)
    g = sns.catplot(x="Genes", y="Expression", col="Cells",
                    data=newdf, 
                    saturation=1,
                    kind="bar", 
                    ci=None, 
                    aspect=.6, 
                    edgecolor="k", 
                    linewidth=1
                    )


    (g.set_axis_labels("", "Expression")
          .set_xticklabels(genes)
          .set_titles("{col_name}")
          .set(ylim=(0, (maxm + 2)))
          .despine(left=True))  

    for ax in g.axes.ravel():
            ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
            for p in ax.patches:
                ax.annotate('%.2f' % p.get_height(), (p.get_x() + p.get_width() / 2., p.get_height()), 

                            ha='center', va='center', xytext=(0, 8), 
                            textcoords='offset points')


In [ ]:
### cell #1

box_layout = widgets.Layout(
        border='solid 1px black',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px',
        height='auto', width='auto')

grid = GridspecLayout(1, 3)
grid.layout = box_layout
# fill it in with widgets
grid[0,0] = specieschoices
grid[0,1] = spacerhuman
grid[0, 2:] = graphs2
grid